In [2]:
import tensorflow as tf 
import pandas as pd
from binance import Client   
from binance import BinanceSocketManager
import time
import os.path
import csv
from time import sleep

In [21]:
def history():
    cryptoFrame = GetPricingHistory('BTCUSDT', '1h', '10y')
    SavePricingHistory(cryptoFrame)
    
history()

NameError: name 'GetPricingHistory' is not defined

In [33]:
def realtimePrice(coin, coinPayment):
    while(True):
     appendRealTimeData(coin, coinPayment)
     time.sleep(1)

realtimePrice("BTC", "USDT")

KeyboardInterrupt: 

In [3]:
#Using this block to instaniate binance API client and using seperate file to just store keys for demo purposes.
%run ./binanceKeys.ipynb
client = Client(apikey, secretKey)

binanceSocketManager = BinanceSocketManager(client)

In [54]:
#https://binance-docs.github.io/apidocs/spot/en/#compressed-aggregate-trades-list KLINE 
def GetPricingHistory(coin, intervalTime, historyTime):
    frame = pd.DataFrame(client.get_historical_klines(coin, intervalTime, historyTime + ' ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
   #Said price to be an int not date format?
   # frame['Time'] = pd.to_datetime(frame['Time'], unit='ms')
    return frame

In [9]:
def SavePricingHistory(pricingHistoryFrame): 
    pricingHistoryFrame.to_csv('data1.csv', encoding='utf-8',index=False)

In [31]:
def appendRealTimeData(coin, coinpayment): 
   combinedCoin = coin + coinpayment
   data = GetRealTimeData(combinedCoin)
   path = coin  + ".csv"
   if(os.path.isfile(path)):
      with open(path, "a", newline='') as fileObj:
        data = data + (coin, coinpayment)
        writer_object = csv.writer(fileObj)
        writer_object.writerow(data)
        fileObj.close()
   else:
     with open(path, "a", newline='') as fileObj:
        writer_object = csv.writer(fileObj)
        headers = ["Time", "Price", "Coin", "Coin Payment"]
        data = data + (coin, coinpayment)
        writer_object.writerow(headers)
        writer_object.writerow(data)
        fileObj.close()
   

In [32]:
def GetRealTimeData(coin):
  coinData = client.get_symbol_ticker(symbol=coin)
  return((time.time() * 1000), coinData["price"]) 
